## 8-Puzzle Game

> ### Overview
The 8-Puzzle game consists of a 3x3 grid with 9 spaces consisting of 8 numbered tiles and one empty space. The game's initial state is a random order with the aim of the goal being to reach the desired goal state. The game is restricted by these following rules:

1. Goal: Go from *Initial State* to *Goal State*
2. Legal Actions: Up, Down, Left,Right
    - Only one tile to be moved at one time.
    - Tiles can only move if adjacent to empty space in current state

The game can be implemented as a search problem with th eplayer searching for a path from the *Initial State* to the *Goal State* by rearranging the tiles.

![Tux, the Linux mascot](assets/8-puzzle.png)

> ### A* Algorithm

Heuristics are *informed* problem-solving methods, this means that they use an esitmation fo the fastest path to the solution. This time efficient solution is based on a simple set of given rules *(i.e in our case being the rules of the puzzle)*

The A* Algorithm is an algorithm that employs heuristic functions to find the path in a state tree with the lowest cost from the *Initial State* to *Goal State*. The algorithm keeps track of all nodes visited and does not explore paths with a higher cost.

> ### Function behind A* Algorithm

The function uses the following function to estimate the cost of every move.

``` f(n) = g(n) + h(n) ```

__*f(n)*__: Total cost of path through node *n* </br>
__*g(n)*__: Sum of costs so far to node *n* </br>
__*h(n)*__: Estimated cost from node *n* to *Goal State*
- Can be calculated using Manhatten Distance from node *n* to *Goal State*.
- Sum of distance of vertical and horizontal steps between two points.

> ### Types of Heuristic Functions
1. *h<sub>1</sub>(n)* : Number of misplaced tiles
2. *h<sub>2</sub>(n)* : sum of Manhatten Distance for every tile from *Current State* to *Goal State*

> ### Reason behind Admission of Fucntions:
1. Both functions do not overestimate the cost of moves to *Goal State*
2. h<sub>1</sub>
    - *Initital State* will always be misplaced and will have to be moved to reach goal state
3. h<sub>2</sub>
    - Every move made by agent will be one step closer to *Goal State*
4. Our goal is to minimize both H<sub>1</sub> and H<sub>2</sub>, bringing us closer to goal state

### 8-Puzzle Solver: Implementing A* Algorithm

In [20]:
import time
from heapq import heappush, heappop
from collections import deque
from math import sqrt

> ### Defining State of Puzzle

In [21]:
class State:
    def __init__(self, state, parent, move, depth, cost, key):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost
        self.key = key
        if self.state:
                self.map = ''.join(str(e) for e in self.state)

    def __eq__(self,other):
        return self.map == other.map

    def __lt__(self,other):
        return self.map < other.map

> ### Inititializing *Goal State*, *Initial State* other parameters

In [22]:
goal_state = [1,2,3,4,5,6,7,8,0]
goal_node = State
init_state = []
puzzle_len = 9
puzzle_side = 0
nodes_expanded = 0
maxDepthReached = 0
maxfringeSize = 0
moves = []

> ### Agent Implementation
#### A* search algorithm
- input parameters:
    1. StartState
        - Input array (i.e. [0,7,5,3,2,4,6,1])
    2. Heuristic Functions *h<sub>n</sub>(n)*

In [23]:
def astar(startState, heuristicFunc):
    global goalNode, maxFringeSize, maxDepthReached
    visited, pQueue = set(), list()
    key = heuristicFunc(startState)
    root = State(startState, None, None, 0, 0, key)
    entry = (key, 0, root)
    heappush(pQueue, entry)
    while pQueue:
        node = heappop(pQueue)
        visited.add(node[2].map)
        if node[2].state == goal_state:
            goalNode = node[2]
            return True, pQueue
        neighbors = expand(node[2])
        for neighbor in neighbors:
            neighbor.key = neighbor.cost + heuristicFunc(neighbor.state)
            entry = (neighbor.key, neighbor.move, neighbor)
            if neighbor.map not in visited:
                heappush(pQueue, entry)
                visited.add(neighbor.map)
                if neighbor.depth > maxDepthReached:
                    maxDepthReached += 1                    
        if len(pQueue) > maxFringeSize:
            maxFringeSize = len(pQueue)
    return False, None

#### Heuristic Functions

In [24]:
def h1(state):
    count = 0
    for i in range(0, 9):
        if not (state.index(i) == goal_state.index(i)) : 
            count+=1
    return count 

In [25]:
def h2(state): 
    return sum(abs(p%puzzle_side - g%puzzle_side) + abs(p//puzzle_side - g//puzzle_side)
               for p,g in ((state.index(i),goal_state.index(i)) 
               for i in range(1, 9))) 

In [31]:
def legalMove(state, position):
    # Function to check that the move is legal
    newState = state[:]
    index = newState.index(0) 
    if position == 'UP': 
        if index not in range(0, puzzle_side):           
            temp = newState[index - puzzle_side]
            newState[index - puzzle_side] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None
    if position == 'DOWN':        
        if index not in range(puzzle_len - puzzle_side, puzzle_len):
            temp = newState[index + puzzle_side]
            newState[index + puzzle_side] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None
    if position == 'LEFT': 
        if index not in range(0, puzzle_len, puzzle_side):
            temp = newState[index - 1]
            newState[index - 1] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None
    if position == 'RIGHT': 
        if index not in range(puzzle_side - 1, puzzle_len, puzzle_side):
            temp = newState[index + 1]
            newState[index + 1] = newState[index]
            newState[index] = temp
            return newState
        else:
            return None

In [32]:
def expand(node):
    global nodesExpanded
    nodesExpanded += 1
    childNodes = []
    childNodes.append(State(legalMove(node.state,'DOWN'),node,'DOWN',node.depth + 1, node.cost + 1, 0)) 
    childNodes.append(State(legalMove(node.state,'LEFT'),node,'LOW',node.depth + 1, node.cost + 1, 0)) 
    childNodes.append(State(legalMove(node.state,'RIGHT'),node,'RIGHT',node.depth + 1, node.cost + 1, 0))
    childNodes.append(State(legalMove(node.state,'UP'),node,'UP',node.depth + 1, node.cost + 1, 0))
    nodes = [child for child in childNodes if child.state]
    return nodes

In [36]:
def get(dataList):
    global puzzle_len, puzzle_side
    data = dataList.split(',') 
    for element in data:
        init_state.append(int(element))
    puzzle_len = len(init_state)                  
    puzzle_side = int(sqrt(puzzle_len))

In [35]:
def backtrack():
    currentNode = goalNode
    while init_state != currentNode.state : 
        moves.insert(0, currentNode.move)
        currentNode = currentNode.parent
    return moves

In [37]:
def output(fringe, time, testNum):
    if fringe:
        global moves
        moves = backtrack()
        print("<-- # SOLUTION DETAILS # -->")
        print("\n Initial State: "+str(init_state))
        print("\n Path to goal: " + str(moves))
        print("\n Cost(Moves): " + str(len(moves)))
        print("\n Node count: " + str(nodesExpanded))
        print("\n Fringe size: " + str(len(fringe)))
        print("\n Tree depth: " + str(goalNode.depth))
        print("\n Run time: " + format(time, '.8f'))
    else:
        print("<-- # NO SOLUTION # -->")
        print("\n Node count: " + str(nodesExpanded))
        print("\n Max fringe: " + str(maxFringeSize))
        print("\n Max tree depth: " + str(maxDepthReached))
        print("\n Run time: " + format(time, '.8f'))

In [38]:
def main():
    heuristic_map = {'h1' : h1,'h2' : h2}
    while True:
        heuristic = input('WELCOME TO 8-PUZZLE SOLVER USING A* ALGORITHM! \
        \n\
        \n--> Please select a heuristic function:\
        \n h1: number of misplaces tiles\
        \n h2: sum of Manhattan distance for every tile from current to goal state \
        \n-- Enter your choice: ')
        if(heuristic == 'h1' or heuristic == 'h2'):
            heuristicFunc = heuristic_map[heuristic]
            break
        print("Please type 'h1' or 'h2'.")
        
        
    data = input('--> Enter puzzle elements: ')    
    get(data)
    function = astar 
    start = time.time()       
    search, fringe = function(init_state, heuristicFunc)  
    stop = time.time()
    if not search : 
        output(fringe, stop-start, 'No_Solution')
    else : 
        output(fringe, stop-start, 'h2')